# DimCustomer – Silver → Gold

**Objetivo:**  
Crear la tabla dimensional `DimCustomer` en la capa Gold a partir de la tabla Silver `AdventureWorks_Silver_Customers`.

**Descripción del proceso:**  
1. Seleccionamos las columnas relevantes de la tabla Silver:
   - `CustomerID` → renombrada como `CustomerKey` (clave subrogada).  
   - `FirstName`, `LastName`, `FullName` → información del cliente.  
   - `ETL_DateInserted` → fecha de ingestión para auditoría.  
2. Creamos una **vista temporal** `tmpCustomer` para facilitar validación y transformaciones posteriores.
3. Finalmente, convertimos la vista en un **Delta Table** en Gold (`Tables/DimCustomer`) lista para análisis y reporting.


In [2]:
%%sql
-- Crear vista temporal tmpCustomer desde Silver
CREATE OR REPLACE TEMP VIEW tmpCustomer AS
SELECT
    CustomerID AS CustomerKey,
    FirstName,
    LastName,
    FullName,
    ETL_DateInserted
FROM AdventureWorks_SilverLayer.AdventureWorks_Silver_Customers;

StatementMeta(, a589913c-a1f1-4256-96ac-c2b654b2655d, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
# Convertir la vista temporal a DataFrame
df = spark.sql("SELECT * FROM tmpCustomer")

# Validación rápida: mostrar las primeras 10 filas
#df.show(10)

# Guardar la tabla DimCustomer en Gold (Delta)
df.write.mode("overwrite") \
       .format("delta") \
       .option("overwriteSchema", "true") \
       .save("Tables/DimCustomer")

StatementMeta(, a589913c-a1f1-4256-96ac-c2b654b2655d, 6, Finished, Available, Finished)